In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
from flask import Flask, request, jsonify
from pyngrok import ngrok
from IPython.display import display, HTML


In [9]:
# Load and preprocess data
titanic_data = pd.read_csv("train.csv")
titanic_data = titanic_data.drop(columns="Cabin", axis=1)
titanic_data['Age'] = titanic_data['Age'].fillna(titanic_data['Age'].mean())
titanic_data['Embarked'] = titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0])

In [10]:
# Convert categorical columns to numeric values
titanic_data['Sex'] = titanic_data['Sex'].map({"male": 0, "female": 1})
titanic_data['Embarked'] = titanic_data['Embarked'].map({"S": 0, "C": 1, "Q": 2})

In [11]:
# Prepare features and target variable
X = titanic_data.drop(columns=["PassengerId", "Name", "Ticket", "Survived"], axis=1)
Y = titanic_data["Survived"]

In [12]:
# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [13]:
# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [14]:
# Evaluate model
train_accuracy = accuracy_score(Y_train, model.predict(X_train))
test_accuracy = accuracy_score(Y_test, model.predict(X_test))
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Training Accuracy: 0.8089887640449438
Test Accuracy: 0.7821229050279329


In [15]:
# Save model
joblib.dump(model, "logistic_regression_model.pkl")

['logistic_regression_model.pkl']

In [16]:
!pip install pyngrok

In [17]:
import subprocess
import os
from pyngrok import ngrok

In [18]:
ngrok.set_auth_token("2kPKm2CBcuAub4jEyEAmEb20zcR_22y8AGWvmNFL2atfnNutv")

In [19]:
os.system("nohup python -m flask run --no reload &")
proc=subprocess.Popen(["ngrok", "http", "5000"])

In [21]:
public_url=ngrok.connect(addr="5000",proto="http")
print(public_url)

NgrokTunnel: "https://8080-183-83-232-151.ngrok-free.app" -> "http://localhost:5000"


In [22]:
model = joblib.load('logistic_regression_model.pkl')

In [ ]:
# Create the Flask app
app = Flask(__name__)

@app.route('/')
def home():
    html_form = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta property="og:title" content="Titanic Survival Prediction">
    <meta property="og:description" content="Predict survival chances on the Titanic based on various features.">
    <meta property="og:image" content="https://example.com/path/to/preview-image.jpg">
    <meta property="og:url" content="http://example.com/">
    <title>Titanic Survival Prediction</title>
    <link rel="icon" href="https://example.com/path/to/favicon.ico" type="image/x-icon">
    <style>
        body {
            background: url('https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/RMS_Titanic_3.jpg/800px-RMS_Titanic_3.jpg') no-repeat center center fixed;
            background-size: cover;
            color: white;
            font-family: Arial, sans-serif;
            text-align: center;
            padding: 20px;
        }
        #predictionForm {
            display: inline-block;
            background-color: rgba(0, 0, 0, 0.7); /* semi-transparent background */
            padding: 20px;
            border-radius: 10px;
            width: 600px; /* Increased width */
            box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.5); /* Shadow for better visibility */
        }
        .form-group {
            display: flex;
            align-items: center;
            margin-bottom: 15px;
        }
        .form-group label {
            flex: 1;
            margin-right: 10px;
            text-align: right;
        }
        .form-group input {
            flex: 2;
            width: 100%;
            padding: 10px;
            border-radius: 5px;
            border: 1px solid #ccc;
        }
        .form-group button {
            background-color: #007bff;
            border: none;
            color: white;
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
            border-radius: 5px;
        }
        .form-group button:hover {
            background-color: #0056b3;
        }
        #predictionResult {
            margin-top: 20px;
            font-size: 18px;
            font-weight: bold;
        }
        h2 {
            color: #000000; /* Black color for the title */
            margin-bottom: 20px;
            font-size: 36px; /* Increase font size here */
            text-align: center;
        }
    </style>
</head>
<body>
    <h2>Titanic Survival Prediction</h2>
    <form id="predictionForm" method="post" action="/predict">
        <div class="form-group">
            <label for="pclass">Pclass:</label>
            <input type="text" id="pclass" name="pclass" required>
        </div>
        <div class="form-group">
            <label for="sex">Sex (0 for male, 1 for female):</label>
            <input type="text" id="sex" name="sex" required>
        </div>
        <div class="form-group">
            <label for="age">Age:</label>
            <input type="text" id="age" name="age" required>
        </div>
        <div class="form-group">
            <label for="sibsp">SibSp:</label>
            <input type="text" id="sibsp" name="sibsp" required>
        </div>
        <div class="form-group">
            <label for="parch">Parch:</label>
            <input type="text" id="parch" name="parch" required>
        </div>
        <div class="form-group">
            <label for="fare">Fare:</label>
            <input type="text" id="fare" name="fare" required>
        </div>
        <div class="form-group">
            <label for="embarked">Embarked (0 for S, 1 for C, 2 for Q):</label>
            <input type="text" id="embarked" name="embarked" required>
        </div>
        <div class="form-group">
            <button type="button" onclick="predictSurvival()">Predict</button>
        </div>
        <p id="predictionResult"></p>
    </form>
    <script>
        function predictSurvival() {
            var xhr = new XMLHttpRequest();
            var url = "/predict";
            var data = new FormData(document.getElementById("predictionForm"));
            xhr.open("POST", url, true);
            xhr.onreadystatechange = function () {
                if (xhr.readyState === 4 && xhr.status === 200) {
                    var response = JSON.parse(xhr.responseText);
                    if (response.prediction !== undefined) {
                        document.getElementById("predictionResult").innerHTML = "Survival Prediction: " + response.prediction;
                    } else if (response.error) {
                        document.getElementById("predictionResult").innerHTML = "Error: " + response.error;
                    } else {
                        document.getElementById("predictionResult").innerHTML = "Unexpected response";
                    }
                }
            };
            xhr.send(data);
        }
    </script>
</body>
</html>

    """
    return html_form

@app.route('/predict', methods=['POST'])
def predict():
    try:
        pclass = int(request.form['pclass'])
        sex = int(request.form['sex'])
        age = float(request.form['age'])
        sibsp = int(request.form['sibsp'])
        parch = int(request.form['parch'])
        fare = float(request.form['fare'])
        embarked = int(request.form['embarked'])
        features = [[pclass, sex, age, sibsp, parch, fare, embarked]]  # Make sure to include all features
        
        # Predict
        prediction = model.predict(features)[0]
        return jsonify({'prediction': int(prediction)})
    
    except Exception as e:
        return jsonify({'error': str(e)})

def run_flask_app():
    app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)

if __name__ == '__main__':
    # Connect ngrok
    public_url = ngrok.connect(addr="5000", proto="http")
    print("Public URL:", public_url)
    
    # Display public URL in notebook
    display(HTML(f"<h2>Open this link in your browser to access the application:</h2><p>{public_url}</p>"))
    
    # Run the Flask app
    try:
        run_flask_app()
    except KeyboardInterrupt:
        ngrok.disconnect(public_url)


Public URL: NgrokTunnel: "https://178d-183-83-232-151.ngrok-free.app" -> "http://localhost:5000"


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Aug/2024 12:07:39] "GET / HTTP/1.1" 200 -
C:\Users\pmoha\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [09/Aug/2024 12:07:56] "POST /predict HTTP/1.1" 200 -
C:\Users\pmoha\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [09/Aug/2024 12:08:39] "POST /predict HTTP/1.1" 200 -
